<a href="https://colab.research.google.com/github/hecshzye/object-detection-tfjs/blob/main/tfod_fabrik_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Realtime Detection using TensorFlow**

# Objective
  - To contruct a Deep Learning algorithm using TensorFlow and Transfer Learning to recognise the connector inside a machinery (🚘)
  - Build a web application where the TensorFlow Js model is running inference over the live webcam with at least 90% accuracy.

# **Tools**

  - TensorFlow Training Custom Object Detector API

    - https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html


  - Dataset labelled using LabelImg Graphical Image annotation

    - https://github.com/tzutalin/labelImg https://tzutalin.github.io/labelImg/

In [1]:
import numpy as np
import pandas as pd
import os
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn

#### Clone TensorFlow Object Detection Models

In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 71912, done.
remote: Total 71912 (delta 0), reused 0 (delta 0), pack-reused 71912
Receiving objects: 100% (71912/71912), 578.88 MiB | 16.70 MiB/s, done.
Resolving deltas: 100% (50848/50848), done.


#### From the TFOD Official Documentation, installing protobuff inside the TFOD research folder

In [3]:
cd /content/models/research

/content/models/research


In [4]:
!protoc object_detection/protos/*.proto --python_out=.

In [5]:
# Installing COCO API
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 14.24 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [6]:
cd cocoapi/PythonAPI

/content/models/research/cocoapi/PythonAPI


In [7]:
!make

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/models/research/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-dIfpci/python3.7-3.7.13=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-dIfpci/python3.7-3.7.13=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/n

In [8]:
cp -r pycocotools /content/models/research

### Install TensorFlow Object Detection API

In [9]:
cd ..

/content/models/research/cocoapi


In [10]:
cd ..

/content/models/research


In [11]:
cp object_detection/packages/tf2/setup.py .

In [12]:
!python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.1 MB 15.1 MB/s 
     |████████████████████████████████| 352 kB 48.1 MB/s 
     |████████████████████████████████| 2.2 MB 33.2 MB/s 
     |████████████████████████████████| 23.4 MB 2.3 MB/s 
     |████████████████████████████████| 237 kB 16.7 MB/s 
     |████████████████████████████████| 1.1 MB 37.8 MB/s 
     |████████████████████████████████| 47.8 MB 51 kB/s 
     |████████████████████████████████| 99 kB 9.5 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 4.9 MB 35.

# Testing the TFOD API

In [13]:
!python object_detection/builders/model_builder_tf2_test.py    

Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-04-15 18:00:57.521203: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0415 18:00:57.844014 139827442939776 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.12s
I0415 18:00:58.305293 139827442939776 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.12s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.68s
I0415 18:00:58.986273 139827442939776 test

In [14]:
 cd /content/fabrik_tfod_demo/pre-trained-models

/content/fabrik_tfod_demo/pre-trained-models


In [15]:
# Download the Pretrained Model (SSD_RESNET101_V1_FPN_640x640)
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2022-04-15 18:12:50--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.140.128, 2a00:1450:400c:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.140.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 386527459 (369M) [application/x-tar]
Saving to: ‘ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet101_v1_fp 100%[===================>] 368.62M  89.7MB/s    in 4.1s    

2022-04-15 18:12:55 (89.7 MB/s) - ‘ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [386527459/386527459]



In [16]:
# Unzip the Pretrained Model
!tar -xvf ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


# Create TF Records 

In [18]:
cd /content/fabrik_tfod_demo

/content/fabrik_tfod_demo


In [19]:
# Create train data:
!python generate_tfrecord.py -x /content/fabrik_tfod_demo/images/train -l /content/fabrik_tfod_demo/annotations/label_map.pbtxt -o /content/fabrik_tfod_demo/annotations/train.record

# Create test data:
!python generate_tfrecord.py -x /content/fabrik_tfod_demo/images/test -l /content/fabrik_tfod_demo/annotations/label_map.pbtxt -o /content/fabrik_tfod_demo/annotations/test.record

Successfully created the TFRecord file: /content/fabrik_tfod_demo/annotations/train.record
Successfully created the TFRecord file: /content/fabrik_tfod_demo/annotations/test.record


# Training the Model

In [25]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [22]:
pip uninstall opencv-python-headless==4.5.5.62

Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.5.5.64.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-65fa80df.so.58.134.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-8ef5c7db.so.58.76.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-9c768859.so.56.70.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-09fe7800.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dis

In [23]:
pip install opencv-python-headless==4.5.2.52

     |████████████████████████████████| 38.2 MB 1.3 MB/s 


In [27]:
!python model_main_tf2.py --model_dir=/content/fabrik_tfod_demo/models/my_ssd_resnet101_v1_fpn --pipeline_config_path=/content/fabrik_tfod_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config

2022-04-15 18:55:08.292793: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0415 18:55:08.343509 140594325870464 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0415 18:55:08.348174 140594325870464 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0415 18:55:08.348340 140594325870464 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0415 18:55:08.536445 140594325870464 deprecation.py:343] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

In [28]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/fabrik_tfod_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config --trained_checkpoint_dir /content/fabrik_tfod_demo/models/my_ssd_resnet101_v1_fpn --output_directory /content/fabrik_tfod_demo/exported_models/my_model

2022-04-15 20:20:20.836789: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0415 20:20:21.037116 139966896240512 deprecation.py:615] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
2022-04-15 20:20:45.505407: W ten